In [1]:
import pandas as pd 
import numpy as np 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
from datetime import date


In [2]:

today = date.today()
d1 = today.strftime("%d/%m/%Y")


In [3]:
dataset = pd.read_csv('output/scrapping_anuncios_shopcar.csv')

## geração de id do shop car

dataset['Id_Anuncio_ShopCar'] = pd.Series(dataset['Link']).str.slice(start=-7)
dataset.reset_index()
dataset.set_index('Id_Anuncio_ShopCar', inplace=True)
dataset.drop(labels=['Unnamed: 0'], axis=1)

## correção do campo data ano/modelo

ano_fabricacao = pd.Series(dataset['Ano']).str.slice(stop=2)
dataset['Ano_Fabricacao'] = pd.to_numeric(ano_fabricacao)
dataset['Ano_Fabricacao'] = np.where(dataset['Ano_Fabricacao'] < 22, 2000 + dataset['Ano_Fabricacao'], 1900 + dataset['Ano_Fabricacao'])

ano_modelo = pd.Series(dataset['Ano']).str.slice(start=3, stop=5)
dataset['Ano_Modelo'] = pd.to_numeric(ano_modelo)
dataset['Ano_Modelo'] = np.where(dataset['Ano_Modelo'] < 22, 2000 + dataset['Ano_Modelo'], 1900 + dataset['Ano_Modelo'])
dataset = dataset.drop(labels=['Ano'], axis=1)
dataset = dataset.drop(labels=['Unnamed: 0'], axis=1)

## correção do campo KM
dataset['KM'] = dataset['KM'].str.replace('Km', '')
dataset['KM'] = dataset['KM'].str.replace('.', '')
dataset['KM'] = pd.to_numeric(dataset['KM'], errors='coerce')

## correção do campo KM
dataset['Preco'] = dataset['Preco'].str.replace('R', '')
dataset['Preco'] = dataset['Preco'].str.replace('$', '')
dataset['Preco'] = dataset['Preco'].str.replace('.', '')
dataset['Preco'] = dataset['Preco'].str.replace(',00', '')
dataset['Preco'] = pd.to_numeric(dataset['Preco'], errors='coerce')

## endereços 
enderecos = pd.Series(dataset['Cidade'])
dataset['Estado_Anuncio'] = enderecos.str.partition('/')[2]
dataset['Endereco_Anuncio'] = enderecos.str.partition('-')[0]

enderecos = enderecos.str.partition('-')[2]
enderecos.str.partition(' - ')
dataset['Bairro_Anuncio'] = enderecos.str.partition('-')[0]

enderecos = enderecos.str.partition('-')[2]
enderecos = enderecos.str.partition('/')
dataset['Cidade_Anuncio'] = enderecos[0]

## marca 
marca = dataset['Link'].str.partition('/')[2]
dataset['Marca'] = marca.str.partition('/')[2].str.partition('/')[2].str.partition('/')[2].str.partition('/')[0]
dataset['Data_Extracao_Dados'] = d1

dataset.to_csv(f'output/tratamento_anuncios_shopcar_{today.strftime("%d%m%Y")}.csv')


## WebMotors

dados_webmotors = pd.read_csv('output/anuncios_webmotors.csv')
dataset_wm = pd.DataFrame()
dataset_wm['Modelo'] = dados_webmotors['Make.Value'] + ' ' + dados_webmotors['Model.Value'] + ' ' + dados_webmotors['Version.Value']
dataset_wm['Cor'] = dados_webmotors['Color.Primary']
dataset_wm[' Combustivel'] = 'Não disponível'
dataset_wm['KM'] = dados_webmotors['Odometer']
dataset_wm['Ano_Fabricacao'] = dados_webmotors['YearFabrication']
dataset_wm['Ano_Modelo'] = dados_webmotors['YearModel']
dataset_wm['Preco'] = dados_webmotors['Price']
dataset_wm['Vendedor'] = 'WebMotors'
dataset_wm['Cidade_Anuncio'] = dados_webmotors['City']
dataset_wm['Estado_Anuncio'] = dados_webmotors['State']
dataset_wm['Marca'] = dados_webmotors['Make.Value']
dataset_wm['Fonte'] = 'WebMotors'
dataset_wm['Data_Extracao_Dados'] = d1
dataset_final = pd.concat([dataset_wm, dataset])
dataset_final

,Modelo,Cor,Combustivel,KM,Ano_Fabricacao,Ano_Modelo,Preco,Vendedor,Cidade_Anuncio,Estado_Anuncio,Marca,Fonte,Data_Extracao_Dados,Combustível,Link,Cidade,Endereco_Anuncio,Bairro_Anuncio
0,JEEP COMPASS 2.0 16V FLEX LONGITUDE AUTOMÁTICO,Prata,Não disponível,41809.0,2019,2020.0,119900.0,WebMotors,Campo Grande,Mato Grosso do Sul (MS),JEEP,WebMotors,10/06/2021,NaN,NaN,NaN,NaN,NaN
1,RENAULT LOGAN 1.0 12V SCE FLEX ZEN MANUAL,Prata,Não disponível,41168.0,2019,2020.0,50990.0,WebMotors,Campo Grande,Mato Grosso do Sul (MS),RENAULT,WebMotors,10/06/2021,NaN,NaN,NaN,NaN,NaN
2,HONDA HR-V 1.8 16V FLEX EX 4P AUTOMÁTICO,Vermelho,Não disponível,92339.0,2016,2016.0,78000.0,WebMotors,Três Lagoas,Mato Grosso do Sul (MS),HONDA,WebMotors,10/06/2021,NaN,NaN,NaN,NaN,NaN
3,RENAULT LOGAN 1.6 EXPRESSION 8V FLEX 4P MANUAL,Preto,Não disponível,187958.0,2016,2017.0,38800.0,WebMotors,Três Lagoas,Mato Grosso do Sul (MS),RENAULT,WebMotors,10/06/2021,NaN,NaN,NaN,NaN,NaN
4,FIAT LINEA 1.8 ESSENCE 16V FLEX 4P AUTOMATIZADO,Cinza,Não disponível,63268.0,2013,2014.0,35800.0,WebMotors,Três Lagoas,Mato Grosso do Sul (MS),FIAT,WebMotors,10/06/2021,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082990,Uno Mille Economy 1.0 Fire 4p.,Branca,NaN,150000.0,2009,2009.0,NaN,Auto Class Veículos,Campo Grande,MS,fiat,NaN,10/06/2021,Flex,https://www.shopcar.com.br/veiculos/fiat/uno-m...,"Av. Bandeirantes, 2820 - Vila Bandeirante - Ca...","Av. Bandeirantes, 2820",Vila Bandeirante
1026471,Uno Mille Economy Way 1.0 Fire 2p.,Branca,NaN,NaN,2010,2011.0,NaN,Gabriel Veículos,São Gabriel do Oeste,MS,fiat,NaN,10/06/2021,Flex,https://www.shopcar.com.br/veiculos/fiat/uno-m...,"Rua Paraná, 1710 - Centro - São Gabriel do Oes...","Rua Paraná, 1710",Centro
1083843,Uno Mille EX 1.0 2p.,Bordo,NaN,165000.0,1997,1998.0,NaN,Auto Class Veículos,Campo Grande,MS,fiat,NaN,10/06/2021,Gasolina,https://www.shopcar.com.br/veiculos/fiat/uno-m...,"Av. Bandeirantes, 2820 - Vila Bandeirante - Ca...","Av. Bandeirantes, 2820",Vila Bandeirante
1084268,Uno Mille Way 1.0 Fire 4p.,Branca,NaN,139000.0,2011,2012.0,NaN,Auto Class Veículos,Campo Grande,MS,fiat,NaN,10/06/2021,Flex,https://www.shopcar.com.br/veiculos/fiat/uno-m...,"Av. Bandeirantes, 2820 - Vila Bandeirante - Ca...","Av. Bandeirantes, 2820",Vila Bandeirante


In [4]:
filtro = dataset_final[(dataset_final.Preco < 40000) & (dataset_final.KM < 50000)].sort_values(by = 'Preco')

In [5]:
qtd_veiculos = dataset.Modelo.count()
qtd_veiculos2 = filtro.Modelo.count()

In [6]:

email_user = '#'
email_password = '#'

recipients = ['#', 'jorgematosltda@gmail.com', 'jvalencarmg@gmail.com'] 
emaillist = [elem.strip().split(',') for elem in recipients]
msg = MIMEMultipart()
msg['Subject'] = f'|SHOP CAR| - {qtd_veiculos} carros extraídos com sucesso! {qtd_veiculos2} parecem ser bons!'
msg['From'] = 'maycon.mota@gmail.com'


html = """\
<html>
  <head></head>
  <body>
<div>
Bom dia, encaminho o relatório contendo todos os veículos do ShopCar extraídos na data de {0}. São filtrados veículos com KM menor que 50000 e menos de R$ 40000.
Boa sorte!
</div>
    {1}
  </body>
</html>
""".format(d1, filtro.to_html())

part1 = MIMEText(html, 'html')
msg.attach(part1)

server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(email_user,email_password)
server.sendmail(msg['From'], emaillist , msg.as_string())

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials t62sm3579803qkc.26 - gsmtp')

In [7]:
filtro

,Modelo,Cor,Combustivel,KM,Ano_Fabricacao,Ano_Modelo,Preco,Vendedor,Cidade_Anuncio,Estado_Anuncio,Marca,Fonte,Data_Extracao_Dados,Combustível,Link,Cidade,Endereco_Anuncio,Bairro_Anuncio
1087855,Chevette SL 1.6,Bege,NaN,7000.0,1979,1979.0,8500.0,Habner Costa Bruno Batista,,,gm-chevrolet,NaN,10/06/2021,Gasolina,https://www.shopcar.com.br/veiculos/gm-chevrol...,Campo Grande - MS,Campo Grande,MS
1740,FIAT PALIO 1.0 MPI EDX 8V GASOLINA 4P MANUAL,Cinza,Não disponível,101.0,1997,1997.0,9900.0,WebMotors,Campo Grande,Mato Grosso do Sul (MS),FIAT,WebMotors,10/06/2021,NaN,NaN,NaN,NaN,NaN
1097,CHEVROLET CORSA 1.6 MPFI GLS 8V GASOLINA 4P MA...,Verde,Não disponível,101.0,1997,1998.0,9900.0,WebMotors,Campo Grande,Mato Grosso do Sul (MS),CHEVROLET,WebMotors,10/06/2021,NaN,NaN,NaN,NaN,NaN
1066313,Gol 1.0 2p.,Cinza,NaN,15800.0,2002,2003.0,10900.0,Revolução Automóveis,Campo Grande,MS,vw-volkswagen,NaN,10/06/2021,Gasolina,https://www.shopcar.com.br/veiculos/vw-volkswa...,"Rua Brilhante, 1925 - Nova Bandeirante - Campo...","Rua Brilhante, 1925",Nova Bandeirante
1087806,Voyage CL 1.6,Prata,NaN,180.0,1994,1994.0,11500.0,Marlon Luiz de Souza,,,vw-volkswagen,NaN,10/06/2021,Gasolina,https://www.shopcar.com.br/veiculos/vw-volkswa...,Dourados - MS,Dourados,MS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077155,Sandero Authentique 1.0 12v,Branca,NaN,46000.0,2018,2019.0,39900.0,Pantanal Veículos,Três Lagoas,MS,renault,NaN,10/06/2021,Flex,https://www.shopcar.com.br/veiculos/renault/sa...,"Rua Munir Thomé,1001 - Centro - Três Lagoas / MS","Rua Munir Thomé,1001",Centro
1079417,Kwid Zen 1.0 12v,Vermelha,NaN,35800.0,2019,2019.0,39900.0,Dois Irmãos Automóveis,Campo Grande,MS,renault,NaN,10/06/2021,Flex,https://www.shopcar.com.br/veiculos/renault/kw...,"Rua Rachid Neder, 401 - Monte Castelo - Campo ...","Rua Rachid Neder, 401",Monte Castelo
1480,HYUNDAI HB20 1.0 COMFORT PLUS 12V FLEX 4P MANUAL,Branco,Não disponível,105.0,2015,2015.0,39990.0,WebMotors,Dourados,Mato Grosso do Sul (MS),HYUNDAI,WebMotors,10/06/2021,NaN,NaN,NaN,NaN,NaN
1090080,Kwid Zen 1.0 12v,Branca,NaN,47831.0,2018,2018.0,39990.0,Guará Seminovos Renault,Campo Grande,MS,renault,NaN,10/06/2021,Flex,https://www.shopcar.com.br/veiculos/renault/kw...,"Av. Ministro João Arinos, 4701 - Chácara Cacho...","Av. Ministro João Arinos, 4701",Chácara Cachoeira
